In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType, TimestampType, LongType
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/01 16:26:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
raw_path = 'data/raw/fhvhv/2021/02/'
pq_path = 'data/pq/fhvhv/2021/02/'
pq_path_temp = pq_path + 'tmp/'
file_name = 'fhvhv_tripdata_2021-02.csv'
url_base = 'https://nyc-tlc.s3.amazonaws.com/trip+data/'
url = url_base + file_name
raw_path_file = raw_path + file_name
pq_path_file = pq_path + file_name

## QUESTION 1

In [144]:
spark.version

'3.0.3'

## QUESTION 2

In [114]:
!mkdir -p raw_path

In [60]:
!wget {url} -O {raw_path_file}

--2022-02-28 21:40:15--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.136.113
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.136.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘data/raw/fhvhv/2021/02/fhvhv_tripdata_2021-02.csv’

data/raw/fhvhv/2021 100%[===================>] 699.83M  33.5MB/s    in 21s     

2022-02-28 21:40:37 (32.6 MB/s) - ‘data/raw/fhvhv/2021/02/fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [165]:
!head {raw_path_file}

In [166]:
schema = StructType([
    StructField("hvfhs_license_num", StringType(), True),
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("SR_Flag", StringType(), True),
])

In [175]:
df = spark.read \
.options(header=True) \
.schema(schema) \
.csv(raw_path_file)

In [176]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [177]:
df.repartition(24).write.parquet(pq_path, mode="overwrite")

In [322]:
!ls -l {pq_path}

total 212936
-rw-r--r-- 1 marcos marcos       0 Mar  1 11:33 _SUCCESS
-rw-r--r-- 1 marcos marcos 9080331 Mar  1 11:33 part-00000-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9080702 Mar  1 11:33 part-00001-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9082175 Mar  1 11:33 part-00002-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9082899 Mar  1 11:33 part-00003-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9082402 Mar  1 11:33 part-00004-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9085799 Mar  1 11:33 part-00005-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9086884 Mar  1 11:33 part-00006-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-rw-r--r-- 1 marcos marcos 9083085 Mar  1 11:33 part-00007-706541da-60e7-49cb-a186-af6240e3268a-c000.snappy.parquet
-r

## QUESTION 3

In [178]:
df = spark.read.parquet(pq_path)

In [179]:
df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('trip_duration', df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long"))

In [180]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|trip_duration|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-------------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null| 2021-02-06|         1319|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null| 2021-02-05|         1130|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null| 2021-02-04|         1484|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null| 2021-02-03|         1375|
|           H

In [181]:
df.write.parquet(pq_path_temp, mode="overwrite")

In [230]:
df = spark.read.parquet(pq_path_temp)

In [231]:
df.filter(df.pickup_date == '2021-02-15').count()

367170

## QUESTION 4

In [212]:
date_of_longest_trip = df.orderBy(df.trip_duration.desc()).take(1)[0].pickup_date
print(date_of_longest_trip)

2021-02-11


Note: orderBy(df._count.desc()) will not work. Error dataframe object has no attribute _count is raised. Use F.col instead
So aggregators are accessed through F.col

## QUESTION 5 (check number of stages)

In [249]:
df \
    .groupBy("dispatching_base_num") \
    .agg(F.count("dispatching_base_num") \
    .alias("_count")) \ 
    .orderBy(F.col("_count").desc()) \
    .show()


+--------------------+-------+
|dispatching_base_num| _count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



## QUESTION 6

In [3]:
df_zones = spark.read.parquet('zones/')

In [4]:
df = spark.read.parquet(pq_path_temp)

Notes: 
Need to use alias in the df name for df_zones in order to distinguish later on when using aggregations
Note: If I do not use alias and just df_zones.LocationID in both joins instead of the F.col with the alias, the Java explodes with memory problems.

Join needs a column object in ==, so use F.col

Need to use first "aggregator" to include that column in aggregations because after group by, only the columns that are used in the group by are kept, in addition to the aggregators

Note: withColumnRenamed needs the column name.

In [5]:
trip = df \
    .join(df_zones.alias("df_zones_pu"), df.PULocationID == F.col("df_zones_pu.LocationID")) \
    .join(df_zones.alias("df_zones_do"), df.DOLocationID == F.col("df_zones_do.LocationID")) \
    .groupBy('PULocationID','DOLocationID') \
    .agg(F.count('DOLocationID').alias('_count'), F.first("df_zones_pu.Zone"), F.first("df_zones_do.Zone")) \
    .withColumnRenamed('first(df_zones_pu.Zone)', 'pickup_zone') \
    .withColumnRenamed('first(df_zones_do.Zone)', 'dropoff_zone') \
    .orderBy(F.col('_count').desc()) \
    .first()

In [7]:
f'{trip["pickup_zone"]} / {trip["dropoff_zone"]}'

'East New York / East New York'

Note: Also I can join after grouping and aggregation, but use 'order by' after joins (otherwise the order is lost). This performs better

In [428]:
trip = df \
    .groupBy('PULocationID','DOLocationID') \
    .agg(F.count('DOLocationID').alias('_count')) \
    .join(df_zones.withColumnRenamed('Zone', 'pickup_zone').alias("df_zones_pu"), df.PULocationID == F.col("df_zones_pu.LocationID")) \
    .join(df_zones.withColumnRenamed('Zone', 'dropoff_zone').alias("df_zones_do"), df.DOLocationID == F.col("df_zones_do.LocationID")) \
    .orderBy(F.col('_count').desc()) \
    .first()

In [429]:
f'{trip["pickup_zone"]} / {trip["dropoff_zone"]}'

'East New York / East New York'